In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import math
import dataframe_image as dfi

In [2]:
#pip install dataframe_image

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
import numpy as np
from datetime import datetime

In [5]:
data = pd.read_csv('data/input/demografico_centro.csv', sep='|')
data0=pd.read_csv('data/input/demografico_centro.csv', sep='|')


In [6]:
data['PRODUCTO'].unique()

array(['DESPENSA', 'LEALTAD'], dtype=object)

In [6]:
data.info(verbose=True,null_counts=True)

<ipython-input-6-83ed814146b6>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  data.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2503760 entries, 0 to 2503759
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   GRUPO              2503760 non-null  object 
 1   CADENA             2503760 non-null  object 
 2   EMAIL              2503760 non-null  object 
 3   EMAIL_1            2097124 non-null  object 
 4   ID_TARJETA         2503760 non-null  object 
 5   MONTO              2503760 non-null  float64
 6   CANAL_COMPRA       2503760 non-null  object 
 7   PRODUCTO           2503760 non-null  object 
 8   GENERO             2351204 non-null  object 
 9   ESTADO             2503760 non-null  object 
 10  ANTIGUEDAD_TH_COM  2503760 non-null  int64  
 11  SUELDO_APROX       2503760 non-null  float64
 12  RANGO_DISPERSION   2503760 non-null  object 
 13  EDAD               2348503 non-null  float64
 14  APP                2503760 non-null  object 
dtypes: float64(3), int64(1), object(

In [8]:
data0.loc[data0['EMAIL']=='S/I','EMAIL']=data0['EMAIL_1']

In [9]:
data['CANAL_COMPRA'].unique()

array(['POS', 'eCommerce'], dtype=object)

In [10]:
#data=data[(data['CANAL_COMPRA']=='eCommerce')| (data['GRUPO']=='WAL MART')]

In [11]:
# limpiamos campo APP quitamos espacios en blanco
data.loc[data['APP']=='Con email      ', 'APP']='Con email'
data.loc[data['APP']=='Sin app        ', 'APP']='Sin app'
data.loc[data['APP']=='Con app        ', 'APP']='Con app'
data.loc[data['APP'].isnull(), 'APP']='Sin app'

In [12]:
#quitamos numermos sin tarjetas 
data=data[data['ID_TARJETA']!='0']
data=data[data['GRUPO']!='OTRO']

In [13]:
data.info(verbose=True,null_counts=True)

<ipython-input-13-83ed814146b6>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  data.info(verbose=True,null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2503760 entries, 0 to 2503759
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   GRUPO              2503760 non-null  object 
 1   CADENA             2503760 non-null  object 
 2   EMAIL              2503760 non-null  object 
 3   EMAIL_1            2097124 non-null  object 
 4   ID_TARJETA         2503760 non-null  object 
 5   MONTO              2503760 non-null  float64
 6   CANAL_COMPRA       2503760 non-null  object 
 7   PRODUCTO           2503760 non-null  object 
 8   GENERO             2351204 non-null  object 
 9   ESTADO             2503760 non-null  object 
 10  ANTIGUEDAD_TH_COM  2503760 non-null  int64  
 11  SUELDO_APROX       2503760 non-null  float64
 12  RANGO_DISPERSION   2503760 non-null  object 
 13  EDAD               2348503 non-null  float64
 14  APP                2503760 non-null  object 
dtypes: float64(3), int64(1), object(

In [14]:
data.loc[(data['GRUPO']=='WAL MART')& (data['CANAL_COMPRA']=='eCommerce'),'GRUPO']='WAL MART Ecommerce'

In [15]:
#data=data[data['GRUPO']!='WAL MART']

In [16]:
data=data[(data['GRUPO']!='SORIANA') &(data['GRUPO']!='CHEDRAUI')]

In [17]:
order=pd.crosstab(data['GRUPO'], columns='count').reset_index()
conteo=order.sort_values(by='count',ascending=False)

In [18]:
conteo[conteo['GRUPO']=='RAPPI']

col_0,GRUPO,count
163,RAPPI,29973


In [19]:
data.loc[(data['GRUPO']!='WAL MART'),'COMERCIO']= data['GRUPO']
data.loc[(data['GRUPO']=='WAL MART') & ((data['CADENA']=='C_ALPRECIO')|(data['CADENA']=='C_BODEGA AURRERA')|(data['CADENA']=='C_MI BODEGA SINALOA DE L')),'COMERCIO']='Bodega'

data.loc[(data['GRUPO']=='WAL MART') & ((data['CADENA']=='C_SUMESA')|(data['CADENA']=='C_BODEGA AURRERA EXPRESS')|(data['CADENA']=='C_SUPERAMA')|(data['CADENA']=='C_WALMART EXP MERIDA OTE')),'COMERCIO']='Express'
data.loc[(data['GRUPO']=='WAL MART') & ((data['CADENA']=='C_CITYCLUB')|(data['CADENA']=='C_CITY MARKET')|(data['CADENA']=='C_SAM S CLUB')),'COMERCIO']='Club de precios'

data.loc[(data['GRUPO']=='WAL MART') & ((data['CADENA']=='C_BAE RIO BRAVENSE')|
                                        (data['CADENA']=='C_BOD L CARDENAS AEROPUE')|
                                        (data['CADENA']=='C_BAE GARCICRESPO')|
                                        (data['CADENA']=='C_BAE CADEREYTA NORTE')|
                                        (data['CADENA']=='C_BAE LA SANTISIMA')|
                                        (data['CADENA']=='C_BAE P DE CHAVARRIA')|
                                        (data['CADENA']=='C_MIB LOS ENCINOS')|
                                        (data['CADENA']=='C_WALMART')|
                                        (data['CADENA']=='C_BAE VILLAS NTE ZUMPANG')|
                                        (data['CADENA']=='C_MIB SALINAS NORTE')|
                                        (data['CADENA']=='C_MIB CHINA')|
                                        (data['CADENA']=='C_BAE P DE LA CONSTITUCI')|
                                        (data['CADENA']=='C_BAE LOS SAUCES')|
                                        (data['CADENA']=='C_SUPERCENTER LAGOS MNO')|
                                        (data['CADENA']=='C_SUPERCENTER APOCADACA')|
                                        (data['CADENA']=='C_BAE BELCANTO')|
                                        (data['CADENA']=='C_MIB CARRANZA SUR')|
                                        (data['CADENA']=='C_BOD HDAS NORESTE')|
                                        (data['CADENA']=='C_MIB CAMALU')|
                                        (data['CADENA']=='C_BAE P DE LA MANZANA')|
                                        (data['CADENA']=='C_BAE CEDIS MERIDA')|
                                        (data['CADENA']=='C_MIB TEPEAPULCO')),'COMERCIO']='SuperMercado'

In [20]:
#data=data[(data['GRUPO']!='CHEDRAUI')&(data['GRUPO']!='SORIANA')]

##### Agrupamos Id_tarjeta y Grupo solo contando para obtener la tabla pivot con 1 y 0
##### 1= tiene compras en el grupo y 0 no tiene compras en el grupo

In [21]:
agrupa = data.groupby(['ID_TARJETA','GRUPO'])['GRUPO'].count()


In [22]:
#agrupa1 = data.groupby(['ID_TARJETA','COMERCIO'])['GRUPO'].count().reset_index()

In [23]:
pivot = agrupa.unstack()

In [24]:
pivot= pivot.fillna(0)

In [25]:
# si es mayor el conteo a 1 se deja 1 y si no se queda en 0
def encode_units(x) :
    if x <=0:
        return 0
    if x>=1:
        return 1

canasta = pivot.applymap(encode_units)

In [26]:
#aplicamos apiori min_suport= frecuencia con la que se encuentra cada grupo a nivel BD
frecuencia = apriori(canasta, min_support=0.01, use_colnames=True)
frecuencia.sort_values(by='support',ascending=False)

,support,itemsets
39,0.742765,(WAL MART)
28,0.261401,(OXXO)
100,0.203288,"(WAL MART, OXXO)"
7,0.118742,(CLIP)
14,0.115544,(FARMACIAS GUADALAJARA)
...,...,...
73,0.010239,"(HEB, FARMACIAS GUADALAJARA)"
24,0.010091,(MCDONALDS)
59,0.010073,"(TIENDAS 3B, CLIP)"
9,0.010045,(CORNERSHOP)


In [27]:
# aplicamos regla de asociacion tomando como metrica el lif en este caso >= 0.09
reglas = association_rules(frecuencia, metric="lift", min_threshold=1)

In [28]:
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(7 ELEVEN),(ALSEA),0.066873,0.053268,0.012726,0.190295,3.572415,0.009163,1.169231
1,(ALSEA),(7 ELEVEN),0.053268,0.066873,0.012726,0.238897,3.572415,0.009163,1.226020
2,(7 ELEVEN),(CLIP),0.066873,0.118742,0.014362,0.214763,1.808647,0.006421,1.122282
3,(CLIP),(7 ELEVEN),0.118742,0.066873,0.014362,0.120949,1.808647,0.006421,1.061517
4,(7 ELEVEN),(FARMACIAS GUADALAJARA),0.066873,0.115544,0.012695,0.189843,1.643040,0.004969,1.091710
...,...,...,...,...,...,...,...,...,...
329,"(MERCADO PAGO, CLIP)","(WAL MART, OXXO)",0.026634,0.203288,0.012177,0.457214,2.249098,0.006763,1.467820
330,(WAL MART),"(OXXO, MERCADO PAGO, CLIP)",0.742765,0.014632,0.012177,0.016395,1.120439,0.001309,1.001792
331,(OXXO),"(WAL MART, MERCADO PAGO, CLIP)",0.261401,0.021251,0.012177,0.046585,2.192101,0.006622,1.026572
332,(MERCADO PAGO),"(WAL MART, OXXO, CLIP)",0.084218,0.040861,0.012177,0.144593,3.538653,0.008736,1.121266


In [30]:
#generamos data Frame de las reglas de asociacion
dataGrafos=reglas[['antecedents','consequents','lift']].reset_index()

In [32]:
#Limpiamos caracteres especiales en DF de reglas de asociacion
dataGrafos["antecedents"] = dataGrafos["antecedents"].astype(str).str.extract(r"\((.*?)\)" , expand=False)
dataGrafos["consequents"] = dataGrafos["consequents"].astype(str).str.extract(r"\((.*?)\)" , expand=False)

In [33]:
dataGrafos["antecedents"] = dataGrafos["antecedents"].replace('{','',regex=True).astype(str)
dataGrafos["antecedents"] = dataGrafos["antecedents"].replace('}','',regex=True).astype(str)

dataGrafos["consequents"] = dataGrafos["consequents"].replace('{','',regex=True).astype(str)
dataGrafos["consequents"] = dataGrafos["consequents"].replace('}','',regex=True).astype(str)

In [34]:
dataGrafos["antecedents"].replace(regex=True, inplace=True, to_replace=r'[^a-zA-Z0-9 ]+', value=r'')
dataGrafos["consequents"].replace(regex=True, inplace=True, to_replace=r'[^a-zA-Z0-9 ]+', value=r'')

In [38]:
dataGrafos[dataGrafos['antecedents']=='WAL MART'].sort_values(by='lift', ascending=False)

,index,antecedents,consequents,lift
249,249,WAL MART,OXXO FARMACIAS SIMILARES,1.169750
225,225,WAL MART,OXXO FARMACIA SAN PABLO,1.169459
267,267,WAL MART,LIVERPOOL OXXO,1.162732
66,66,WAL MART,FARMACIAS SIMILARES,1.154727
291,291,WAL MART,OXXO TIENDAS 3B,1.150434
213,213,WAL MART,FARMACIA DEL AHORRO FARMACIAS GUADALAJARA,1.144989
219,219,WAL MART,OXXO FARMACIA DEL AHORRO,1.142490
207,207,WAL MART,OXXO COSTCO,1.139958
316,316,WAL MART,OXXO CLIP FARMACIAS GUADALAJARA,1.136858
303,303,WAL MART,OXXO TIENDAS SUPER PRECIO,1.135029


##### Generamos grafos 

In [ ]:
G = nx.DiGraph()

In [ ]:
for row in dataGrafos.iterrows():
    G.add_edge(row[1]['antecedents'],
                  row[1]['consequents'])

In [ ]:
nx.draw_circular(G, node_color='lightblue',
                edge_color='gray',
                 font_size=10,
                with_labels=True, width=3)
plt.savefig("imagen.png")
plt.show()


#### agregamos los sub grupos para el caso de Wal mart

##### Quitamos tiendas en expecifico en caso de ser necesario 

In [ ]:
#dataWalmart=  data[(data['GRUPO']!='SORIANA') & (data['GRUPO']!='CHEDRAUI')]

In [ ]:
dataGrafosCadena=dataGrafos

In [ ]:
dataGrafosCadena

#### Asignamos las variables a analisar base= tiendas Antecedentes
#### comercios = Tiendas consecuentes

In [ ]:
#comercios base
base='WAL MART'
base1=''
base2=''
base3=''
base4=''

In [ ]:
# COMERCIOS COINCIDENCIA
comercio1='RAPPI'
comercio2='TIENDAS 3B'
comercio3=''
comercio4=''
comercio5=''

In [ ]:
id_tarjeta='ID_TARJETA'

In [ ]:
#generamos las listas de las variables base y comercios 
lista1=[base]
lista2=[comercio1,comercio2,comercio3,comercio4]

##### Recorremos la tabla de Grafos ya con limites de lift para generar la tabla con la que se trabajara, es decir tabla pivote para todos los calculos y referencias 

In [ ]:
largoA=len(dataGrafosCadena)-len(dataGrafosCadena)
largoB=len(dataGrafosCadena)-len(dataGrafosCadena) +1
while largoA<=len(dataGrafosCadena):
    for i in dataGrafosCadena.iloc[largoA:largoB,:].index:
        for i in lista1:
            for j in lista2:
                dataGrafosCadena.loc[(dataGrafosCadena['antecedents']==i) &(dataGrafosCadena['consequents']==j),'flag']=1
    largoA+=1
    largoB+=1

In [ ]:
dataL=dataGrafosCadena[dataGrafosCadena['flag']>=1]


In [ ]:
dataL

In [ ]:
dataLimpia=dataL[['antecedents','consequents','lift']]

In [ ]:

dataLimpia=dataLimpia[['antecedents','consequents','lift']]

In [ ]:
dataLimpia

In [ ]:
G = nx.DiGraph()
for row in dataLimpia.iterrows():
    G.add_edge(row[1]['antecedents'],
                  row[1]['consequents'])

In [ ]:
nx.draw_circular(G, node_color='lightblue',
                edge_color='gray',
                 font_size=9,
                with_labels=True, width=2)
plt.show()

In [ ]:
tabla_monto=pd.pivot_table(data,values='MONTO', columns='GRUPO',index=['ID_TARJETA','APP'] )

In [ ]:
transacciones=pd.pivot_table(data,values='MONTO', columns='GRUPO',index=['ID_TARJETA','APP'] ).reset_index()

#### Recorremos la tabla pivote para obtener las combinbaciones en base a la tabla referencia llamada dataLimpia 

In [ ]:
pos=len(dataLimpia)-len(dataLimpia)
pos1=len(dataLimpia)-len(dataLimpia) +1
while pos<=len(dataLimpia):
    for i in dataLimpia.iloc[pos:pos1,:].index:
        for colum in tabla_monto.loc[:,[dataLimpia['antecedents'][i],dataLimpia['consequents'][i]]].columns: 
            tabla_monto.loc[(tabla_monto[dataLimpia['antecedents'][i]]>=1) & (tabla_monto[dataLimpia['consequents'][i]]>=1),'combinacion' + str(pos)]=dataLimpia['antecedents'][i] + '-' +dataLimpia['consequents'][i]
    pos+=1
    pos1+=1
    

In [ ]:
tabla_monto=tabla_monto.reset_index()

In [ ]:
tabla_monto

#### Generamos variable dummy para calcular registros con APP

In [ ]:
dummy = pd.get_dummies(tabla_monto['APP'])
dummy = dummy.replace({0: None})
tabla_monto_fin = pd.concat([tabla_monto,dummy],axis=1)

In [ ]:
'''
pos=len(dataLimpia)-len(dataLimpia)
pos1=len(dataLimpia)-len(dataLimpia) +1
comb=0
while pos<=len(dataLimpia):
    for i in dataLimpia.iloc[pos:pos1,:].index:
        for colum in tabla_monto_fin.loc[:,[dataLimpia['antecedents'][i],dataLimpia['consequents'][i]]].columns: 
            tabla_monto_fin.loc[(tabla_monto_fin[dataLimpia['antecedents'][i]]>=1) & (tabla_monto_fin[dataLimpia['consequents'][i]]>=1),'segmento' + str(pos)]=1
            tabla_monto_fin['segmento' + str(pos)]=tabla_monto_fin['segmento' + str(pos)].fillna(0)
            tabla_monto_fin['segmento' + str(pos)]=tabla_monto_fin['segmento' + str(pos)].astype(int)
            

    pos+=1
    pos1+=1
    comb+=1
'''    

##### Generamos las agrupaciones por cada combinacion para obtener los totales 

In [ ]:
pos=len(dataLimpia)-len(dataLimpia)
#pos = lasgo de la dfata - largo data para bucle
pos1=len(dataLimpia)-len(dataLimpia) +1
comb=0

while pos<=len(dataLimpia):
    for i in dataLimpia.iloc[pos:pos1,:].index:
        var=str(["comb" + str(comb)])
        # locals utilizamos para generar una variable que cambie su nombre una para cada bucle o una para cada combinacion
        locals()["comb" + str(comb)] = tabla_monto_fin.groupby(['combinacion'+ str(comb)]).agg({dataLimpia['antecedents'][i]:'sum',dataLimpia['consequents'][i]:'sum',id_tarjeta:'count','Con app':'sum','Sin app':'sum','Con email':'sum'}).reset_index()
        locals()["comb" + str(comb)] = locals()["comb" + str(comb)].rename(columns={'combinacion'+ str(comb):'combinacion',dataLimpia['antecedents'][i]:'monto_walmart',dataLimpia['consequents'][i]:'monto_otros','ID_TARJETA':'tarjetahabientes'})
       
       
        
        
    

    pos+=1
    pos1+=1
    comb+=1
    
    

#### Segmentamos la Data con los clienetes que Tienen APP 

In [ ]:
con_app=tabla_monto_fin[tabla_monto_fin['APP']=='Con app']

##### Se realiza mismo agrupacion pero solo con los clientes con app

In [ ]:

pos=len(dataLimpia)-len(dataLimpia)
pos1=len(dataLimpia)-len(dataLimpia) +1
comb=0

while pos<=len(dataLimpia):
    for i in dataLimpia.iloc[pos:pos1,:].index:
        locals()["comb_con" + str(comb)] = con_app.groupby(['combinacion'+ str(comb)]).agg({dataLimpia['antecedents'][i]:'sum',dataLimpia['consequents'][i]:'sum',id_tarjeta:'count','Con app':'sum','Sin app':'sum','Con email':'sum'}).reset_index()
        locals()["comb_con" + str(comb)] = locals()["comb_con" + str(comb)].rename(columns={'combinacion'+ str(comb):'combinacion',dataLimpia['antecedents'][i]:'monto_walmart',dataLimpia['consequents'][i]:'monto_otros','ID_TARJETA':'tarjetahabientes'})
        
    pos+=1
    pos1+=1
    comb+=1
    

In [ ]:
comb1

##### Generamos tabla Final asi como el formato

In [ ]:
dataAgrupa = pd.concat([comb0,comb1], axis=0)
dataAgrupaApp=pd.concat([comb_con0,comb_con1], axis=0)

In [ ]:
dataAgrupa['monto_otros']=dataAgrupa['monto_otros'].round()
dataAgrupa['monto_walmart']=dataAgrupa['monto_walmart'].round()

dataAgrupaApp['monto_otros']=dataAgrupaApp['monto_otros'].round()
dataAgrupaApp['monto_walmart']=dataAgrupaApp['monto_walmart'].round()

In [ ]:
dataAgrupa['monto_otros']=dataAgrupa['monto_otros'].map(lambda x:format(x,','))
dataAgrupa['monto_walmart']=dataAgrupa['monto_walmart'].map(lambda x:format(x,','))
dataAgrupa['tarjetahabientes']=dataAgrupa['tarjetahabientes'].map(lambda x:format(x,','))
dataAgrupa['Con app']=dataAgrupa['Con app'].map(lambda x:format(x,','))
dataAgrupa['Sin app']=dataAgrupa['Sin app'].map(lambda x:format(x,','))
dataAgrupa['Con email']=dataAgrupa['Con email'].map(lambda x:format(x,','))

dataAgrupaApp['monto_otros']=dataAgrupaApp['monto_otros'].map(lambda x:format(x,','))
dataAgrupaApp['monto_walmart']=dataAgrupaApp['monto_walmart'].map(lambda x:format(x,','))
dataAgrupaApp['tarjetahabientes']=dataAgrupaApp['tarjetahabientes'].map(lambda x:format(x,','))
dataAgrupaApp['Con app']=dataAgrupaApp['Con app'].map(lambda x:format(x,','))
dataAgrupaApp['Sin app']=dataAgrupaApp['Sin app'].map(lambda x:format(x,','))
dataAgrupaApp['Con email']=dataAgrupaApp['Con email'].map(lambda x:format(x,','))

In [ ]:
dataLimpia['combinacion']= dataLimpia['antecedents'] + '-' +dataLimpia['consequents']

In [ ]:
merge=pd.merge(dataLimpia,dataAgrupa, on='combinacion')
mergeApp=pd.merge(dataLimpia,dataAgrupaApp, on='combinacion')

In [ ]:
final=merge[['combinacion','lift','monto_walmart','monto_otros','tarjetahabientes','Con app','Sin app','Con email']]
finalApp=mergeApp[['combinacion','lift','monto_walmart','monto_otros','tarjetahabientes','Con app']]


In [ ]:
#dfi.export(final , 'final_test.png' )


In [ ]:
finalApp

In [ ]:
#a=tabla_monto_fin[['ID_TARJETA','segmento0']]

In [ ]:
#data0=data0[data0['ID_TARJETA']!='0']
#data0=data0[data0['GRUPO']!='OTRO']

In [ ]:
#data1=data0.groupby(['ID_TARJETA','ESTADO','RANGO_DISPERSION','GENERO'])['ANTIGUEDAD_TH_COM','SUELDO_APROX','EDAD'].max().reset_index()

In [ ]:
#test=pd.merge(data1,transacciones, on='ID_TARJETA', how='inner')

In [ ]:
#test1=pd.merge(test,a, on='ID_TARJETA', how='inner')

In [ ]:
#test1.info(verbose=True,null_counts=True)

In [ ]:
comercio2

In [ ]:
tabla_monto.loc[tabla_monto[comercio1].notnull(),'segmento0']=1
tabla_monto.loc[tabla_monto[comercio1].isnull(),'segmento0']=0

tabla_monto.loc[tabla_monto[comercio2].notnull(),'segmento1']=1
tabla_monto.loc[tabla_monto[comercio2].isnull(),'segmento1']=0


In [ ]:
a=tabla_monto[['ID_TARJETA','segmento0','segmento1']]

In [ ]:
data0=data0[data0['ID_TARJETA']!='0']
data0=data0[data0['GRUPO']!='OTRO']
#data0=data0[data0['GRUPO']!='WAL MART']

In [ ]:
data1=data0.groupby(['ID_TARJETA','EMAIL','ESTADO','RANGO_DISPERSION','GENERO'])['ANTIGUEDAD_TH_COM','SUELDO_APROX','EDAD'].max().reset_index()

In [ ]:
test=pd.merge(data1,transacciones, on='ID_TARJETA', how='inner')

In [ ]:
test1=pd.merge(test,a, on='ID_TARJETA', how='inner')

In [ ]:
test1.info(verbose=True,null_counts=True)

In [ ]:

test1.to_csv('data/output/demograficos.csv',sep='|', index = False)